In [ ]:
import cv2
import mediapipe as mp
import time

In [ ]:
class PoseDetector():
    def __init__(self, mode=False, upBody=False, smooth=True, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.upBody = upBody
        self.smooth=smooth
        self.detectionCon=detectionCon
        self.trackCon = trackCon
        
        self.mpPose = mp.solutions.pose
        self.pose = self.mpPose.Pose(self.mode, self.upBody, self.smooth, self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        
    def findPose(self, img, draw=True):  
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.pose.process(imgRGB)
       # print(results)
       #print(results.pose_landmarks)
        if self.results.pose_landmarks: # if landmarks are detected then
            if draw: 
                self.mpDraw.draw_landmarks(img, self.results.pose_landmarks, self.mpPose.POSE_CONNECTIONS)
                
        return img
    
    def Landmarks(self, img, draw=True,lm_num=0):
        lmList = []
        if self.results.pose_landmarks:
            for id, lm in enumerate(self.results.pose_landmarks.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                if id == lm_num:
                    lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx,cy), 5, (255,0,255), cv2.FILLED)
        return lmList
                
            

In [ ]:
def main():
    cap = cv2.VideoCapture('posEstimationvideos/pexels-marc-espejo-6548148.mp4')
    pTime = 0
    detector = PoseDetector()
    
    while True:
        success, img = cap.read()
        img = detector.findPose(img)
        lmList = detector.Landmarks(img, 7)
        if len(lmList) > 0:
            print(lmList)
        #print(img.shape)
        #img = cv2.resize(img, (1000,700))


        cTime = time.time()
        fps = 1/(cTime-pTime)
        pTime = cTime

        cv2.putText(img, str(int(fps)), (70,50), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0), 3 )
        cv2.imshow('Image', img)

        cv2.waitKey(1)   

In [ ]:
if __name__ == '__main__':
    main()